In [25]:
from minio import Minio
import os
from dbt.cli.main import dbtRunner
from dbt.cli.main import Manifest

from io import BytesIO

In [2]:
access_key = "kDZHuGN0Wwa3DzSA84rZ"
secret_key = "F6w8XW60S816HFY01wqx9Wxyn8Cydi2igcCw7pR1"

In [3]:

bucket_name = "airflow-metadata"

client = Minio(
    endpoint="192.168.0.147:30091",
    access_key=access_key,
    secret_key=secret_key,
    secure=False
)

In [4]:

if not client.bucket_exists(bucket_name):
    client.make_bucket(bucket_name)


In [7]:

object_name = "dbt_parse"


dbt_runner = dbtRunner()
result = dbt_runner.invoke(["parse", "--project-dir", "./testing"])



19:06:30  Running with dbt=1.7.9


19:06:30  Registered adapter: postgres=1.7.9
19:06:30  Performance info: testing\target\perf_info.json


b'\xde\x00\x1b\xa5nodes\x86\xd9 model.testing.my_first_dbt_model\xde\x00\'\xa8database\xa8postgres\xa6schema\xa6public\xa4name\xb2my_first_dbt_model\xadresource_type\xa5model\xacpackage_name\xa7testing\xa4path\xbeexample\\my_first_dbt_model.sql\xb2original_file_path\xd9%models\\example\\my_first_dbt_model.sql\xa9unique_id\xd9 model.testing.my_first_dbt_model\xa3fqn\x93\xa7testing\xa7example\xb2my_first_dbt_model\xa5alias\xb2my_first_dbt_model\xa8checksum\x82\xa4name\xa6sha256\xa8checksum\xd9@0f0a39768ca58b1f5b1e85acbb46843e16a97d88db12e7a25d1533b2e8f77b36\xa6config\xde\x00\x17\xa7enabled\xc3\xa5alias\xc0\xa6schema\xc0\xa8database\xc0\xa4tags\x90\xa4meta\x80\xa5group\xc0\xacmaterialized\xa5table\xb4incremental_strategy\xc0\xacpersist_docs\x80\xa9post-hook\x90\xa8pre-hook\x90\xa7quoting\x80\xaccolumn_types\x80\xacfull_refresh\xc0\xaaunique_key\xc0\xb0on_schema_change\xa6ignore\xb7on_configuration_change\xa5apply\xa6grants\x80\xa8packages\x90\xa4docs\x82\xa4show\xc3\xaanode_color\xc0\xa8c

In [15]:

res = client.put_object(
    bucket_name,
    object_name,
    BytesIO(result.result.to_msgpack()),
    length=-1,
    content_type="application/octet-stream",
    part_size=10*1024*1024
)


In [30]:
data = client.get_object(bucket_name, object_name)

In [31]:
manifest = Manifest.fromø_msgpack(data.read())

In [39]:

for name, node in manifest.nodes.items():
    for dep in node.depends_on.nodes:
        print(dep)

model.testing.my_first_dbt_model
model.testing.my_first_dbt_model
model.testing.my_first_dbt_model
model.testing.my_second_dbt_model
model.testing.my_second_dbt_model
